In [16]:
%pip install pydantic 
%pip install email-validator

import functools
from pydantic import BaseModel, EmailStr, Field, field_validator
from typing import Callable, List, Optional

class Book(BaseModel):
    title: str
    author: str
    year: int
    available: bool
    categories: List[str] = Field(default_factory=list)

    @field_validator('categories', mode='before')
    def validate_categories(cls, value):
        if not all(isinstance(cat, str) and cat for cat in value):
            raise ValueError("Каждая категория должна быть не пустой строкой")
        return value

class User(BaseModel):
    name: str
    email: EmailStr
    membership_id: str
    
class BookNotAvailable(Exception):
    def __init__(self, message="Книга не доступна"):
        self.message = message
        super().__init__(self.message)
    
def log_operation(func: Callable) -> Callable:
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        book = args[1] if len(args) > 1 else None
        print(
          f"Executing '{func.__name__}' for book '{book.title}'" if book else f"Executing '{func.__name__}'"
        )
        result = func(*args, **kwargs)
        print(
          f"Completed '{func.__name__}' for book '{book.title}'" if book else f"Completed '{func.__name__}'"
        )
        return result
    return wrapper

class Library(BaseModel):
    books: List[Book] = Field(default_factory=list)
    users: List[User] = Field(default_factory=list)

    @log_operation
    def add_book(self, book: Book) -> None:
        self.books.append(book)

    def find_book(self, title: str) -> Optional[Book]:
        for book in self.books:
            if book.title == title:
                return book
        return None

    @log_operation
    def is_book_borrow(self, book: Book) -> bool:
        if not book.available:
            raise BookNotAvailable(f"Книга '{book.title}' не доступна для проката.")
        return False

    @log_operation
    def return_book(self, book: Book) -> None:
        book.available = True

    def total_books(self) -> int:
        return len(self.books)
    
    
book1 = Book(
    title="1984", 
    author="George Orwell", 
    year=1949, 
    available=True, 
    categories=["Dystopian", "Fiction"]
    )

book2 = Book(
    title="The Great Gatsby", 
    author="F. Scott Fitzgerald", 
    year=1925, 
    available=False, 
    categories=["Classic", "Fiction"]
    )

user1 = User(name="Alice", email="alice@example.com", membership_id="001")

library = Library()

library.add_book(book1)
library.add_book(book2)

print(f"Книг в библиотеке: {library.total_books()}")

found_book = library.find_book("1984")
if found_book:
    print(f"Найденная книга: {found_book.title} от автора {found_book.author}")
    

try:
    library.is_book_borrow(book2)
except BookNotAvailable as e:
    print(e)


library.return_book(book2)

try:
    library.is_book_borrow(book2)
    print(f"Книга '{book2.title}' доступна для проката.")
except BookNotAvailable as e:
    print(e)


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Executing 'add_book' for book '1984'
Completed 'add_book' for book '1984'
Executing 'add_book' for book 'The Great Gatsby'
Completed 'add_book' for book 'The Great Gatsby'
Книг в библиотеке: 2
Найденная книга: 1984 от автора George Orwell
Executing 'is_book_borrow' for book 'The Great Gatsby'
Книга 'The Great Gatsby' не доступна для проката.
Executing 'return_book' for book 'The Great Gatsby'
Completed 'return_book' for book 'The Great Gatsby'
Executing 'is_book_borrow' for book 'The Great Gatsby'
Completed 'is_book_borrow' for book 'The Great Gatsby'
Книга 'The Great Gatsby' доступна для проката.
